**Update 07.08.20: The features that are related to the target gene will be calculated seperately.** Thus, the following code is NOT using the target sequences that apear here. 

**The notebook that includes this calculation is "Target Gene Features.ipnb"**

## import libraries:

In [1]:
from Bio.Data.CodonTable import unambiguous_dna_by_id
from Bio.Data import CodonTable
from itertools import chain
import numpy as np
from Bio.SeqUtils.CodonUsage import CodonsDict
from Bio.Data import CodonTable
from collections import Counter
import re
import gzip
from Bio import SeqIO
import pandas as pd
from os import chdir
from statistics import mean, pstdev
from scipy.stats.mstats import gmean
from CAI import CAI, relative_adaptiveness
from operator import itemgetter, attrgetter

# CAI calculation references:

* the following package, written by Benjamin Lee, is availble online: https://github.com/Benjamin-Lee/CodonAdaptationIndex
it is an implementation of Sharp and Li's 1987 formulation of the codon adaption index.
* the source code (reading is recommend!): https://github.com/Benjamin-Lee/CodonAdaptationIndex/blob/master/CAI/CAI.py
* reference to the paper written by Benjamin Lee, discussing the implementation of this package: https://joss.theoj.org/papers/10.21105/joss.00905
* documentation of this package: https://buildmedia.readthedocs.org/media/pdf/cai/latest/cai.pdf

attention: the latest version is 1.0.3.

In [2]:
# pip install CAI

### Methods for Degenerated Codons (ENC calculation)
#### http://peterjc.github.io/wiki/Degenerated_Codons

In [3]:
print(CodonTable.standard_dna_table)

Table 1 Standard, SGC0

  |  T      |  C      |  A      |  G      |
--+---------+---------+---------+---------+--
T | TTT F   | TCT S   | TAT Y   | TGT C   | T
T | TTC F   | TCC S   | TAC Y   | TGC C   | C
T | TTA L   | TCA S   | TAA Stop| TGA Stop| A
T | TTG L(s)| TCG S   | TAG Stop| TGG W   | G
--+---------+---------+---------+---------+--
C | CTT L   | CCT P   | CAT H   | CGT R   | T
C | CTC L   | CCC P   | CAC H   | CGC R   | C
C | CTA L   | CCA P   | CAA Q   | CGA R   | A
C | CTG L(s)| CCG P   | CAG Q   | CGG R   | G
--+---------+---------+---------+---------+--
A | ATT I   | ACT T   | AAT N   | AGT S   | T
A | ATC I   | ACC T   | AAC N   | AGC S   | C
A | ATA I   | ACA T   | AAA K   | AGA R   | A
A | ATG M(s)| ACG T   | AAG K   | AGG R   | G
--+---------+---------+---------+---------+--
G | GTT V   | GCT A   | GAT D   | GGT G   | T
G | GTC V   | GCC A   | GAC D   | GGC G   | C
G | GTA V   | GCA A   | GAA E   | GGA G   | A
G | GTG V   | GCG A   | GAG E   | GGG G   | G
--+---------

In [4]:
def altcodons(codon, table):
    """List codons that code for the same aminonacid / are also stop. 
    Given a codon, the output is a list of the alternative (synonymous) codons.

    @param codon
    @table code table id (1 is standard)
    @return list of codons

    """
    tab = unambiguous_dna_by_id[table]

    if codon in tab.stop_codons:
        return tab.stop_codons

    try:
        aa = tab.forward_table[codon]
    except:
        return []

    return [k for (k, v) in tab.forward_table.items()
            if v == aa]


In [5]:
def degeneration(codon, table):
    """Determine how many codons code for the same amino acid / are also stop (degeneration degree of the input codon)

    @param codon the codon
    @param table code table id
    @return the number of codons also coding for the amino acid codon codes for

    """
    return len(altcodons(codon, table))

In [6]:
def define_table():
    """Define the table that maps each amino acid to its synonymus codons (without stop codons).
    @return a dictionary, where the keys are amino acids and the values are the codons.
    
    """
    
    standard_table = CodonTable.unambiguous_dna_by_name["Standard"]
    reverse_genetic_code = standard_table.back_table.copy()
    # aa = reverse_genetic_code.keys()
    # codons = reverse_genetic_code.values()

    for aa in reverse_genetic_code.keys():
        curr_amino_acid = aa
        primary_codon = reverse_genetic_code[aa]
        syno_codons = altcodons(primary_codon, 1)
        reverse_genetic_code[curr_amino_acid] = syno_codons

    reverse_genetic_code.pop(None, None)
    return reverse_genetic_code


In [7]:
tab = define_table()
print(tab)

{'K': ['AAA', 'AAG'], 'N': ['AAT', 'AAC'], 'T': ['ACT', 'ACC', 'ACA', 'ACG'], 'R': ['CGT', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'S': ['TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC'], 'I': ['ATT', 'ATC', 'ATA'], 'M': ['ATG'], 'Q': ['CAA', 'CAG'], 'H': ['CAT', 'CAC'], 'P': ['CCT', 'CCC', 'CCA', 'CCG'], 'L': ['TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG'], 'E': ['GAA', 'GAG'], 'D': ['GAT', 'GAC'], 'A': ['GCT', 'GCC', 'GCA', 'GCG'], 'G': ['GGT', 'GGC', 'GGA', 'GGG'], 'V': ['GTT', 'GTC', 'GTA', 'GTG'], 'Y': ['TAT', 'TAC'], 'C': ['TGT', 'TGC'], 'W': ['TGG'], 'F': ['TTT', 'TTC']}


In [8]:
def count_codons(gene_seq):
    """For each codon in gene_seq, the function calculates the number of apperances of the codon. 
    The stop codons are not counted for because we don't use them for ENC calculation.
    
    @param gene_seq the sequence of the gene (not string type!)
    @return a dictionary where the keys are codons and the values are the number of appereances for the codon in the seq.
    if the input is illegal sequence (stop codon in the middle of the orf) than the output is empty dictionary.
    
    """
    #counts = collections.defaultdict(int)
    error_dict = CodonsDict.copy() #empty dictionary
    error_dict = {x:np.NaN for x in error_dict}
    counts = CodonsDict.copy()
    [counts.pop(k) for k in altcodons("TAA",1)] #remove stop codons
    [error_dict.pop(k) for k in altcodons("TAA",1)] #remove stop codons
    for i in range(0,len(gene_seq),3):
        codon = gene_seq[i:i+3]
        #if there is a stop codon in the middle of the sequence, return empty dictionary, else continue count:
        if codon in altcodons("TAA",1): return error_dict #illegal sequence
        else: counts[codon] += 1
    return counts

In [9]:
def codon_relative_freq(gene_seq):
    """For each codon, the function calculates its relative frequency,
    i.e the number of apperances of the codon,
    divided by the number of appereances of the amino-acid that the codon encodes.
    if the codon is not in the sequence, its relative frequency is set to NaN.
    
    @param gene_seq the sequence of the gene (not string type!)
    @return a dictionary where the keys are codons and the values are the relative frquencies.

    """
    
    counts = count_codons(gene_seq)
    gene_seq
    genetic_code = unambiguous_dna_by_id[1].forward_table #keys = codons; values = Amino acids
    reverse_genetic_code = define_table() #keys = Amino acids, values = codons
    protein_seq = gene_seq.translate() 
    return {codon: float(counts[codon])/protein_seq.count(genetic_code[codon]) if protein_seq.count(genetic_code[codon])!=0 else np.NaN 
            for codon in counts.keys()}

### functions that calculate the demand of codons in the transcriptom (for NTE calculation)

In [10]:
def codon_demand_per_gene(counts,gene_id,gene_seq,gene_mrna):
    """The codon demand for gene 'g' is defined by Ng*Mg: 
    * the number of apperances of the codon (Ng, unique for the codon and the gene). 
    * mRNA abundance of the gene (Mg, unique for the gene).
    for each codon in gene_seq, the function calculates the demand per gene (Ng*Mg) and updates it to the counts dictionary
    
    @param counts a dictionary where the keys are the codons and the values are the demands
    @param gene_id the gene ORF name
    @param gene_seq the sequence of the gene (seq type, not string type)
    @param gene_mrna the mRNA abundance of the gene
    @return the updated 'counts' dictionary.
    """
    #counts = CodonsDict.copy() #ng*mg 
    for i in range(0,len(gene_seq),3):
        codon = gene_seq[i:i+3]
        try:
            counts[codon] += gene_mrna
            #counts[codon] += mRNA_data[gene_id]
        except:
            counts[codon] += 0
    return counts

In [11]:
def codon_demand(sequences):
    """Total codon demand is defined by how many times it appears in the transcriptom (all the genes of the organism)
    The formula for codon i: demand_{i} = \sum_{g}(Ng_i*Mg)
    """
    u_counts = CodonsDict.copy() #ng*mg 
    for seq in sequences:
        u_counts = codon_demand_per_gene(u_counts,seq.id,seq.seq,seq.mrna)
    return u_counts

## functions that calculate the CUB measures:

In [12]:
def ENC_calc(gene_seq):
    """The effective number of codons (ENC; Wright, 1990) measures the deviation from equal use of synonymous codons.
    The ENC is the number of codons that, if used equally, would generate the level of codon bias observed.
    ENC takes values from 20.0 (maximum bias) to 61.0 (no bias).
    Article: https://eclass.uoa.gr/modules/document/file.php/D473/%CE%92%CE%B9%CE%B2%CE%BB%CE%B9%CE%BF%CE%B3%CF%81%CE%B1%CF%86%CE%AF%CE%B1/DNA%20Composition/Wright_1990.pdf
    
    The formulas:
    ENC =  2+ \frac{9}{F2} + \frac{1}{F3} + \frac{5}{F4} + \frac{3}{F6} 
    f_k = \sum_j^k(H_j) for k=2,3,4,6 (degeneration degree)
    H_j = \frac{n*\sum_i^k(p_i^2)-1}{n-1} for i=1,...k (synonymus codon of amino acid aa)
    
    Fk is the average homozygosity (Hj) of amino acids 'j' with degeneration degree k.
    Hj is the homozygosity of amino acid 'j' and is calculated from the square codon frequencies. 
    
    @param gene_seq the sequence (not a string type!)
    @return ENC index
    """
    
    gene_seq = gene_seq[0:-3] #remove stop codon:
    freqs = dict() #sum of frequencies for each AA
    dd = dict() # this dictionary will group the AA with the same degeneration degrees. the values will be the homozygosities
    dd["1"] = []
    dd["2"] = []
    dd["3"] = []
    dd["4"] = []
    dd["6"] = []
    relative_freq = codon_relative_freq(gene_seq) #a dictionary of relative frequencies for each codon
    codon_counter = count_codons(gene_seq) #a dictionary of number of appereances for each codon
    reverse_genetic_code = define_table() # define the table that maps each amino acid to its synonymus codons (withoue STOP)
    for aa in reverse_genetic_code.keys():
        sum_p_power = 0
        pow_sum = 0
        n = 0
        synonyms_codons = reverse_genetic_code[aa] 
        for codon in synonyms_codons:
            sum_p_power += pow(relative_freq[codon],2)
            n += codon_counter[codon]
            deg_aa = str(degeneration(codon,1))
        try:
            effective_number_for_aa = ((n*sum_p_power)-1)/(n-1) #Homozygosity of AA (Hj)
        except ZeroDivisionError:
            #Rarely used aa are those aa for which either the numerator or denominator of "effective_number_for_aa" is 0.
            #if this occurs they should be treated as absent (place NaN)
            effective_number_for_aa = np.NaN
        freqs[aa] = effective_number_for_aa
        dd[deg_aa].append(effective_number_for_aa);

    # When there are no amino acids in a synonymous family, Nc is not calculated as the gene is either too short
    # or has extremely skewed amino acid usage.
    F2 = np.nanmean(dd["2"])
    F3 = np.nanmean(dd["3"])
    F4 = np.nanmean(dd["4"])
    F6 = np.nanmean(dd["6"])

    #if isoleucine (I) is missing, then F3 should be calculated as the mean of F2 and F4.
    if np.isnan(F3): 
        F3 = (F2+F4)/2

    #If any of the other S-F types are completely missing or 'rarely used' then the gene is probably too short
    #(e.g., less than 61 codons) to accurately measure SCU bias, or exhibits extremely skewed aa usage. 
    if (F2==0) | (F3==0) | (F4==0) | (F6==0): 
        ENC = np.NaN
    else:
        ENC = 2+ (9/F2) + (1/F3) + (5/F4) + (3/F6)
    
    #The value of ENC can be greater than 61 if the observed codon usage pattern is more uniform than expected by chance.
    #Such a rare event is most likely when aa composition is very extreme (e.g., such that the Fs are obtained
    #by averaging over very few aa), and the gene is very short. In these cases, the value of ENC should be revised to 61. 
    if ENC>61:
        ENC=61
    
    return ENC

In [13]:
def tAI_profile(sequence, weights):
    """creates a list of the imported weights for the sequence, counting all codons.""" 
    sequence = sequence.upper()
    sequence = [sequence[i : i + 3] for i in range(0, len(sequence), 3)]
    sequence_weights = []
    for codon in sequence:
        sequence_weights.append(weights[codon])
    return sequence_weights

In [14]:
def calc_NTE_weights(gene_data,mRNA_data,tAI_weights):
    """NTE is normalized translation efficiency. it it defined as the supply/demand ratio,
    which is why the weight of each codon is calculated as tAI/demand."""
    for i in range(len(gene_data)):
        try:
            gene_data[i].mrna = mRNA_data[gene_data[i].id]
        except:
            gene_data[i].mrna = None
    cu = codon_demand(gene_data) #demand of each codon
    cu_norm = {k: v/max(cu.values()) for k, v in cu.items()} #normalized demand of each codon
    nte = {k: v/cu_norm[k] for k, v in tAI_weights.items()} #the supply/demand ratio.
    # we assume that the supply is correlative to the tAI becaus we dont have the tRNA levels of each codon in the cell.
    NTE_weights = {k: v/max(nte.values()) for k, v in nte.items()} #normalized NTE for each codon
    return NTE_weights

In [15]:
def NTE_profile(sequence, weights):
    """create a list of the calculated weights for the sequence, counting all codons."""
    sequence = sequence.upper()
    sequence = [sequence[i : i + 3] for i in range(0, len(sequence), 3)] #cut the sequence to codons
    sequence_weights = []
    for codon in sequence:
        sequence_weights.append(weights[codon])
    return sequence_weights

In [16]:
def calc_RCBS_per_gene(gene_seq):
    """The function calculates the RCBS of the gene, which is defined as: RCBS = geomean([d_xyz+1])-1
    For each codon we calculate the RCBS weight, which is defined as d_xyz = [f(x,y,z)-f1(x)f2(y)f3(z)]/[f1(x)f2(y)f3(z)]
    where f(x,y,z) is the normalized frequency of the codon xyz (number of apperances divided with length of the sequence),
    and f1(x) is the normalized frequency of the nucleotide x in the first position of the codons in the seq.
    From these weights (which are unique for the gene), we create a profile of d_xyz+1, calculate the geometric mean,
    and substract 1. This leads to the RCBS index of the sequence.
    Original article: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2646356/"""

    #counts = CodonsDict.copy() #ng*mg
    codons_list = (gene_seq[i:i + 3].upper() for i in range(0, len(gene_seq), 3))
    codons_list = list(codons_list) #a flat list of all the codons in the sequence. the length of the list is len(seq)/3
    counts = Counter(codons_list) #dictionary sub-class

    L = len(gene_seq)/3
    Fxyz = {k: v/L for k, v in counts.items()} #the normalized frquency of the codon
    
    F1 = dict.fromkeys(['A','C','G','T']) #here we insert the observed freq of each nuc in the first codon position
    F1['A'] = len(list((val for val in codons_list if re.findall("\AA", val))))/L
    F1['C'] = len(list((val for val in codons_list if re.findall("\AC", val))))/L
    F1['G'] = len(list((val for val in codons_list if re.findall("\AG", val))))/L
    F1['T'] = len(list((val for val in codons_list if re.findall("\AT", val))))/L
    
    F2 = dict.fromkeys(['A','C','G','T']) #here we insert the observed freq of each nuc in the second codon position
    F2['A'] = len(list((val for val in codons_list if val[1]=='A')))/L
    F2['C'] = len(list((val for val in codons_list if val[1]=='C')))/L
    F2['G'] = len(list((val for val in codons_list if val[1]=='G')))/L
    F2['T'] = len(list((val for val in codons_list if val[1]=='T')))/L
    
    F3 = dict.fromkeys(['A','C','G','T']) #here we insert the observed freq of each nuc in the third codon position
    F3['A'] = len(list((val for val in codons_list if re.findall("A$", val))))/L
    F3['C'] = len(list((val for val in codons_list if re.findall("C$", val))))/L
    F3['G'] = len(list((val for val in codons_list if re.findall("G$", val))))/L
    F3['T'] = len(list((val for val in codons_list if re.findall("T$", val))))/L
    
    ### calculaute the RCBS weights per codon (d_xyz):
    RCBS_weights = CodonsDict.copy()
    for codon in RCBS_weights.keys():
        x = codon[0]
        y = codon[1]
        z = codon[2]
        try:
            RCBS_weights[codon] = (Fxyz[codon]-F1[x]*F2[y]*F3[z])/(F1[x]*F2[y]*F3[z]) #d_xyz definition
        except KeyError:
            RCBS_weights[codon] = 0 #the codon does not apprear in the sequence, hence its weight is set to zero.
    
    ## calculate the profile, for RCBS calculation: 
    sequence_profile = [] 
    for codon in codons_list:
        sequence_profile.append(1+RCBS_weights[codon])
    
    #Now sequence_weights is a list of (1+d_xyz), so we can calculate the geoman-1, thus achive RCBS index.
    RCBS_gene = gmean(sequence_profile)-1
    return RCBS_gene

In [17]:
def calc_RCA_weights(ref_seq):
    """The function calculates the RCA weights per codon in the reference genome. The weight of codon xyz is defined as:
    RCA_xyz = [f(x,y,z)]/[f1(x)f2(y)f3(z)]
    where f(x,y,z) is the normalized frequency of the codon xyz (number of apperances divided with length of the sequences),
    and f1(x) is the normalized frequency of the nucleotide x in the first position of the codons in the seq.
    From these weights (which are calculated based on whole genome), we create a profile of RCA_xyz (see function 
    RCA_profile). This leads to the RCA index of the sequence.
    Original article: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2885275/"""
    
    ## count the number of apperances of each codon in the reference set of genes:
    sequences = (list((str(sequence[i:i + 3].upper())) for i in range(0, len(sequence), 3)) for sequence in ref_seq)
    codons_list = list(sequences) # flat list of all codons (to be used for counting)
    codons_list = list(chain(*codons_list)) # flat list of all codons (to be used for counting)
    counts = Counter(codons_list)

    L = len(codons_list)
    CHIxyz = {str(k): v/L for k, v in counts.items()} #the normalized frquency of the codon
    
    CHI1 = dict.fromkeys(['A','C','G','T']) #here we insert the observed freq of each nuc in the first codon position
    CHI1['A'] = len(list((val for val in codons_list if re.findall("\AA", val))))/L
    CHI1['C'] = len(list((val for val in codons_list if re.findall("\AC", val))))/L
    CHI1['G'] = len(list((val for val in codons_list if re.findall("\AG", val))))/L
    CHI1['T'] = len(list((val for val in codons_list if re.findall("\AT", val))))/L
    
    CHI2 = dict.fromkeys(['A','C','G','T']) #here we insert the observed freq of each nuc in the second codon position
    CHI2['A'] = len(list((val for val in codons_list if val[1]=='A')))/L
    CHI2['C'] = len(list((val for val in codons_list if val[1]=='C')))/L
    CHI2['G'] = len(list((val for val in codons_list if val[1]=='G')))/L
    CHI2['T'] = len(list((val for val in codons_list if val[1]=='T')))/L
    
    CHI3 = dict.fromkeys(['A','C','G','T']) #here we insert the observed freq of each nuc in the third codon position
    CHI3['A'] = len(list((val for val in codons_list if re.findall("A$", val))))/L
    CHI3['C'] = len(list((val for val in codons_list if re.findall("C$", val))))/L
    CHI3['G'] = len(list((val for val in codons_list if re.findall("G$", val))))/L
    CHI3['T'] = len(list((val for val in codons_list if re.findall("T$", val))))/L
    
    ### calculaute the RCBS weights per codon (d_xyz):
    RCA_weights = CodonsDict.copy()
    for codon in RCA_weights.keys():
        x = codon[0]
        y = codon[1]
        z = codon[2]
        try:
            RCA_weights[codon] = (CHIxyz[codon])/(CHI1[x]*CHI2[y]*CHI3[z]) #d_xyz definition
        except KeyError:
            RCA_weights[codon] = 0 #the codon does not apprear in the reference sequence, hence its weight is set to zero.
    return RCA_weights
    

In [18]:
def RCA_profile(sequence,weights):
    """create a list of the calculated weights for the sequence, counting all codons."""
    sequence = sequence.upper()
    sequence = [sequence[i : i + 3] for i in range(0, len(sequence), 3)] #cut the sequence to codons
    sequence_weights = []
    for codon in sequence:
        sequence_weights.append(weights[codon])
    return sequence_weights

# Data loading and CUB calculation:

## data loading:

In [19]:
directory = r'/home/karinsio/my_project_dir/iGEM/Genes_Features_seq'
chdir(directory)

In [20]:
standard_table = CodonTable.unambiguous_dna_by_name["Standard"]
print(standard_table.back_table)

{'K': 'AAG', 'N': 'AAT', 'T': 'ACT', 'R': 'CGT', 'S': 'TCT', 'I': 'ATT', 'M': 'ATG', 'Q': 'CAG', 'H': 'CAT', 'P': 'CCT', 'L': 'TTG', 'E': 'GAG', 'D': 'GAT', 'A': 'GCT', 'G': 'GGT', 'V': 'GTT', 'Y': 'TAT', 'C': 'TGT', 'W': 'TGG', 'F': 'TTT', None: 'TAA'}


In [21]:
def import_orf_coding():
### get sequence data
    with gzip.open(genetic_encoding_file, 'rt') as data:
        gene_data_list = [seq for seq in SeqIO.parse(data, 'fasta') if len(seq)% 3 == 0]
    return gene_data_list;

In [22]:
def import_refernce_seq_wholeGenome():
    """get sequences of all the yeast genome, to use as refercnce for calculation of CAI and RCA"""
    with gzip.open(genetic_encoding_file, 'rt') as data:
        #ref is only the sequence of the genes that are divided by 3
        ref = [seq.seq for seq in SeqIO.parse(data, 'fasta') if len(seq)% 3 == 0]
    return ref;

In [23]:
def import_PA_data():
    """read the Protein Abundance (PA) file and extract the PA of each protein (dictionary)"""
    
    PA_filename = '4932-GPM_2012_09_Saccharomyces_cerevisiae.txt'
    PA_data = pd.read_csv(PA_filename,header=11,delimiter='\t') #DataFrame
    PA_data = PA_data.drop(columns=['#internal_id','raw_spectral_count']).rename(columns={"string_external_id": "ORF"})
    PA_data=PA_data.set_index('ORF').to_dict()
    PA_data=PA_data['abundance'] #dictionay, the keys are ORF names and the values are abundance level of protein.
    PA_data = {k[5:]: v for k, v in PA_data.items()} #cut the "4932." chars from the protein ORF name.
    return PA_data

In [24]:
PA_data = import_PA_data()

In [25]:
def import_reference_seq_HEG(PA_data):
    """import the highly expressed genes sequences (2% of the genome with the higest protein abundance)
    to use as referance seq for calculation of CAI and RCA."""
    
    sorted_PA = sorted(list(PA_data.items()), key=itemgetter(1), reverse=True) #sorted list from high to low PA
    highest_PA = sorted_PA[0:round(len(sorted_PA)*0.02)] #list of tuples (ORF,PA) containing 20% of the genes with highly protain abundance 
    HEG_id = list(map(itemgetter(0), highest_PA)) #list of the ORF of the highly expressed genes (HEG)
    ### get the sequences of the highly expressed genes (HEG, account for 2% of the genome, have highest PA),
    ### to use as reference for calculation of CAI:
    with gzip.open(genetic_encoding_file, 'rt') as data:
        #ref is only the sequence of the genes that are included in the list of HEG_id
        ref_HEG = [seq.seq for seq in SeqIO.parse(data, 'fasta') if HEG_id.count(seq.id) == 1]
    return ref_HEG

In [26]:
## coding sequence for each gene in yeast:
genetic_encoding_file = r'orf_coding_all.fasta.gz'
gene_data = import_orf_coding()

## referance sequences: 
ref_seq_wholeGenome = import_refernce_seq_wholeGenome()
ref_seq_HEG = import_reference_seq_HEG(PA_data)

In [27]:
def import_tAI_weights():
    """read the tAI file and extract the weight for each codon (dictionary)"""
    
    tAI_file = r'1-s2.0-S0092867410003193-mmc2.xls'
    xls = pd.ExcelFile(tAI_file)
    df1 = pd.read_excel(xls, 'tAI',header=5,skiprows=(70,71))
    tAI_df = df1.drop(columns=['anti-codon','Human',' E. coli K12','C. elegance','D. melanogaster','Aeropyrum pernix ']).rename(columns={"S. Cerevisiae": "tAI_weight"}).set_index('Codon')
    tAI_weights = tAI_df.to_dict()
    tAI_weights=tAI_weights['tAI_weight'] #dictionary, the keys are codons and the values are weights for tAI calculation.
    return tAI_weights

In [28]:
def import_mrna_data():
    """read the mRNA file and extract the demand of each codon, and then the NTE for each codon (dictionary)"""
    
    mRNA_filename = 'GSE75897_RiboZero_RPKMs.txt'
    mRNA_data = pd.read_csv(mRNA_filename,header=None,delimiter='\t')
    mRNA_data=mRNA_data.set_index(0).to_dict()
    mRNA_data=mRNA_data[1] #dictionay, the keys are ORF names and the values are mRNA levels (in RPKM)
    return mRNA_data

In [29]:
mRNA_data = import_mrna_data()

## cub calculation:

In [30]:
def CUB_calc(gene_data_list,ref_seq_wholeGenome,ref_seq_HEG,mRNA_data,PA_data):
    """The Main Function!
    @return a DataFrame of the genes and their codon usage bias analysis.
    """
    winSize = 48 #window size in nucleotides
    winStep = 1 #window step in nucleotides
    numWin = 100 #number of windows in the gene. 
    # if gene length is smaller than 150 nuc, than the window value will be placed with nan
    
    # calculate the weights for CAI: the reference genome is all the yeast genome
    CAI_weights_wholeGenome = relative_adaptiveness(sequences=ref_seq_wholeGenome)
    # calculate the weights for CAI: the reference genome is the highly expressed genes (HEG) with the highest PA: 
    CAI_weights_HEG = relative_adaptiveness(sequences=ref_seq_HEG)
    #calculate the tAI weights:
    tAI_weights = import_tAI_weights()
    # calculate the NTE weights (based on the whole organism):
    NTE_weights = calc_NTE_weights(gene_data_list,mRNA_data,tAI_weights)
    #calculate the weights for RCA: the reference genome is all the yeast genome
    RCA_weights_wholeGenome = calc_RCA_weights(ref_seq=ref_seq_wholeGenome)
    #calculate the weights for RCA: the reference genome is the highly expressed genes (HEG) with the highest PA: 
    RCA_weights_HEG = calc_RCA_weights(ref_seq=ref_seq_HEG)    
    
    all_genes = dict()
    for gene in gene_data_list:
        curr_gene = dict()
         ### gene sequence
        curr_gene['gene_sequence'] = str(gene.seq)
        ### length of nuc sequence
        curr_gene['length'] = len(str(gene.seq))
        ### gene ID 
        curr_gene['ORF'] = str(gene.id)
        ### gene mRNA levels:
        try:
            curr_gene['mRNA_RPKM'] = mRNA_data[str(gene.id)]
        except:
            curr_gene['mRNA_RPKM'] = np.NaN
        ### gene Protein abundance (PA) levels:
        try:
            curr_gene['PA'] = PA_data[str(gene.id)]
        except:
            curr_gene['PA'] = np.NaN
                
        ### gene ENC:
        curr_gene['ENC'] = ENC_calc(gene.seq)
        ### gene CAI, ref=wholeGenome: 
        cai_wholeGenome = CAI(str(gene.seq), weights=CAI_weights_wholeGenome)
        curr_gene['CAI_index_wholeGenome'] = cai_wholeGenome[0];
        #curr_gene['CAI_profile_wholeGenome'] = cai_wholeGenome[1];
        curr_gene['CAI_STD_wholeGenome'] = float(pstdev(cai_wholeGenome[1])); #standard deviation
        curr_gene['CAI_first_50_codons_wholeGenome'] = float(gmean(cai_wholeGenome[1][0:50]));
        curr_gene['CAI_last_50_codons_wholeGenome'] = float(gmean(cai_wholeGenome[1][-50:])); 
        ### gene CAI, ref=HEG: 
        cai_HEG = CAI(str(gene.seq), weights=CAI_weights_HEG)
        curr_gene['CAI_index_HEG'] = cai_HEG[0];
        #curr_gene['CAI_profile_HEG'] = cai_HEG[1];
        curr_gene['CAI_STD_HEG'] = float(pstdev(cai_HEG[1]));
        curr_gene['CAI_first_50_codons_HEG'] = float(gmean(cai_HEG[1][0:50]));
        curr_gene['CAI_last_50_codons_HEG'] = float(gmean(cai_HEG[1][-50:])); 
        ### gene tAI:
        tai_output = tAI_profile(str(gene.seq), weights=tAI_weights)
        curr_gene['tAI_index'] = float(gmean(tai_output))
        #curr_gene['tAI_profile'] = tai_output
        curr_gene['tAI_STD'] = float(pstdev(tai_output))
        curr_gene['tAI_first_50_codons'] = float(gmean(tai_output[0:50]));
        curr_gene['tAI_last_50_codons'] = float(gmean(tai_output[-50:]));        
        ### gene NTE:
        nte_output = NTE_profile(str(gene.seq), weights=NTE_weights)
        curr_gene['NTE_index'] = float(gmean(nte_output))
        #curr_gene['NTE_profile'] = nte_output
        curr_gene['NTE_STD'] = float(pstdev(nte_output))
        curr_gene['NTE_first_50_codons'] = float(gmean(nte_output[0:50]));
        curr_gene['NTE_last_50_codons'] = float(gmean(nte_output[-50:]));   

        ### gene RCBS: 
        curr_gene['RCBS'] = calc_RCBS_per_gene(str(gene.seq))
        ### gene RCA, ref = wholeGenome:
        rca_wholeGenome = RCA_profile(str(gene.seq),weights=RCA_weights_wholeGenome)
        curr_gene['RCA_index_wholeGenome'] = float(gmean(rca_wholeGenome))
        #curr_gene['RCA_profile_wholeGenome'] = rca_wholeGenome
        curr_gene['RCA_STD_wholeGenome'] = float(pstdev(rca_wholeGenome))
        curr_gene['RCA_first_50_codons_wholeGenome'] = float(gmean(rca_wholeGenome[0:50]));
        curr_gene['RCA_last_50_codons_wholeGenome'] = float(gmean(rca_wholeGenome[-50:]));   
        ### gene RCA, ref = wholeGenome:
        rca_HEG = RCA_profile(str(gene.seq),weights=RCA_weights_HEG)
        curr_gene['RCA_index_HEG'] = float(gmean(rca_HEG))
        #curr_gene['RCA_profile_HEG'] = rca_HEG
        curr_gene['RCA_STD_HEG'] = float(pstdev(rca_HEG))
        curr_gene['RCA_first_50_codons_HEG'] = float(gmean(rca_HEG[0:50]));
        curr_gene['RCA_last_50_codons_HEG'] = float(gmean(rca_HEG[-50:]));
        
        ########## windowing:
        gene_seq = gene.seq
        for i in range(1,numWin+1):
            win = gene_seq[i:i+winSize] #seq
            if len(win)==winSize:
                ### window CAI, ref=wholeGenome: 
                caiWin_wholeGenome = CAI(str(win), weights=CAI_weights_wholeGenome)
                curr_gene['CAI_index_win'+str(i)+'_wholeGenome'] = caiWin_wholeGenome[0];
                ### window CAI, ref=HEG: 
                caiWin_HEG = CAI(str(win), weights=CAI_weights_HEG)
                curr_gene['CAI_index_win'+str(i)+'_HEG'] = caiWin_HEG[0];
                ### window tAI:
                tai_output_win = tAI_profile(str(win), weights=tAI_weights)
                curr_gene['tAI_index_win'+str(i)] = float(gmean(tai_output_win))
                ### window NTE:
                tai_output_win = NTE_profile(str(win), weights=NTE_weights)
                curr_gene['NTE_index_win'+str(i)] = float(gmean(tai_output_win))
                ### window RCBS: 
                curr_gene['RCBS_win'+str(i)] = calc_RCBS_per_gene(str(win))
                ### window RCA, ref = wholeGenome:
                rcaWin_wholeGenome = RCA_profile(str(win),weights=RCA_weights_wholeGenome)
                curr_gene['RCA_index_win'+str(i)+'_wholeGenome'] = float(gmean(rcaWin_wholeGenome))
                ### window RCA, ref = wholeGenome:
                rcaWin_HEG = RCA_profile(str(win),weights=RCA_weights_HEG)
                curr_gene['RCA_index_win'+str(i)+'_HEG'] = float(gmean(rcaWin_HEG))
            else:
                curr_gene['CAI_index_win'+str(i)+'_wholeGenome']=np.NaN
                curr_gene['CAI_index_win'+str(i)+'_HEG']=np.NaN
                curr_gene['tAI_index_win'+str(i)]=np.NaN
                curr_gene['NTE_index_win'+str(i)]=np.NaN
                curr_gene['CAI_index_win'+str(i)+'_wholeGenome']=np.NaN
                curr_gene['RCBS_win'+str(i)]=np.NaN
                curr_gene['RCA_index_win'+str(i)+'_wholeGenome']=np.NaN
                curr_gene['RCA_index_win'+str(i)+'_HEG'] = np.NaN
        
        ## insert all the features of the current gene into the final table:
        all_genes[gene.id] = curr_gene
        
    
    gene_data_df = pd.DataFrame.from_dict(all_genes, orient = 'index').reset_index().rename(columns={'index':'ORF'})
    return gene_data_df


In [31]:
gene_data_df = CUB_calc(gene_data,ref_seq_wholeGenome,ref_seq_HEG,mRNA_data,PA_data)
# gene_data_df.to_csv(r'CUB_results_ver2.csv')
gene_data_df.to_excel(r'CUB_results_final.xlsx',engine='xlsxwriter')

/home/karinsio/my_project_dir/my_project_env/lib/python3.6/site-packages/ipykernel_launcher.py:51: RuntimeWarning: Mean of empty slice
/home/karinsio/my_project_dir/my_project_env/lib/python3.6/site-packages/ipykernel_launcher.py:50: RuntimeWarning: Mean of empty slice
/home/karinsio/my_project_dir/my_project_env/lib/python3.6/site-packages/ipykernel_launcher.py:52: RuntimeWarning: Mean of empty slice
/home/karinsio/my_project_dir/my_project_env/lib/python3.6/site-packages/ipykernel_launcher.py:53: RuntimeWarning: Mean of empty slice


In [32]:
gene_data_df

,ORF,gene_sequence,length,ORF,mRNA_RPKM,PA,ENC,CAI_index_wholeGenome,CAI_STD_wholeGenome,CAI_first_50_codons_wholeGenome,...,RCBS_win99,RCA_index_win99_wholeGenome,RCA_index_win99_HEG,CAI_index_win100_wholeGenome,CAI_index_win100_HEG,tAI_index_win100,NTE_index_win100,RCBS_win100,RCA_index_win100_wholeGenome,RCA_index_win100_HEG
0,YAL001C,ATGGTACTGACGATTTATCCTGACGAACTCGTACAAATAGTGTCTG...,3483,YAL001C,37.80,11.60,52.499762,0.734169,0.250486,0.759264,...,2.750944,1.222763,0.787172,0.707072,0.178779,0.241931,0.027772,1.879886,1.032415,0.405449
1,YAL002W,ATGGAGCAAAATGGCCTTGACCACGACAGCAGATCTAGCATCGATA...,3825,YAL002W,22.17,7.27,50.928649,0.742818,0.239425,0.714702,...,3.061334,1.092761,0.320760,0.523649,0.093294,0.189421,0.046871,2.131717,0.605164,0.262181
2,YAL003W,ATGGCATCCACCGATTTCTCCAAGATTGAAACTTTGAAACAATTAA...,621,YAL003W,4621.86,3029.00,26.669428,0.825756,0.190030,0.836821,...,2.118197,1.197576,1.908536,0.640120,0.117762,0.193914,0.025949,2.458412,1.115684,0.371074
3,YAL004W,ATGGGTGTCACCAGCGGTGGCCTTAACTTCAAAGATACCGTCTTCA...,648,YAL004W,NaN,26.80,34.435740,0.674925,0.240300,0.659022,...,3.178232,0.991119,0.402804,0.637584,0.123598,0.297283,0.043497,2.118197,0.883690,0.389291
4,YAL005C,ATGTCAAAAGCTGTCGGTATTGATTTAGGTACAACATACTCGTGTG...,1929,YAL005C,NaN,1117.00,28.341978,0.823721,0.195552,0.837872,...,2.391503,1.175160,1.734433,0.665937,0.123181,0.187328,0.027386,2.358922,0.987152,0.399857
5,YAL007C,ATGATCAAATCTACAATTGCTCTACCCTCTTTCTTCATTGTTTTAA...,648,YAL007C,212.42,255.00,51.760655,0.777593,0.223742,0.778706,...,2.517394,1.121169,0.682821,0.769939,0.215059,0.304178,0.034379,2.517394,0.990837,0.456100
6,YAL008W,ATGACTTTGGCTTTTAATATGCAACGGTTGGTGTTTCGTAATTTGA...,597,YAL008W,49.96,72.70,51.661854,0.714422,0.260086,0.661697,...,3.178232,1.069899,0.499223,0.567410,0.081334,0.183980,0.054649,3.382807,0.434412,0.195369
7,YAL009W,ATGGAGCCAGAGAGCATAGGCGATGTGGGGAACCATGCCCAGGATG...,780,YAL009W,34.96,37.20,58.167277,0.690552,0.262954,0.579445,...,2.507514,0.969335,0.366193,0.540432,0.087199,0.082978,0.019178,2.507514,0.713440,0.237107
8,YAL010C,ATGCTACCCTATATGGACCAAGTACTAAGGGCATTTTATCAGAGCA...,1482,YAL010C,16.67,6.19,50.675034,0.739047,0.240760,0.664406,...,2.236818,1.068921,0.505618,0.803427,0.370478,0.380276,0.044353,1.721829,0.926069,0.619003
9,YAL011W,ATGCCTGCTGTCTTGAGAACCAGGTCCAAAGAATCCTCTATAGAGC...,1878,YAL011W,17.40,404.00,53.630576,0.719718,0.246476,0.718603,...,0.721549,1.114492,0.566084,0.498507,0.037608,0.328014,0.045826,0.981409,0.912412,0.336965
